In [2]:
# Milestone 5: Advanced Model Development - Part 1
# Import libraries and set up environment

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import joblib


In [ ]:
# Part 2: Load and prepare data (robust target detection with fallback)

# Example: Replace this with your cleaned dataset file path
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("cleaned_dynamic_pricing.csv")
# Normalize column names to be robust to variants
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')

# Candidate target names (priority order)
target_candidates = [
    'optimal_price', 'optimalprice', 'optimal_price_usd',
    'optimal_price_inr', 'price', 'fare', 'predicted_price', 'pred_price', 'dynamic_price'
]

# Allow the user to override mapping by setting a dict named `COLUMN_MAP` earlier in notebook
# e.g. COLUMN_MAP = {'target': 'optimal_price'}
mapped_target = None
if 'COLUMN_MAP' in globals() and isinstance(COLUMN_MAP, dict):
    # attempt to use mapping if provided (normalize mapping key)
    user_target = COLUMN_MAP.get('target')
    if user_target and user_target in data.columns:
        mapped_target = user_target

# Find the first matching candidate in the dataframe
target_col = mapped_target
if target_col is None:
    for c in target_candidates:
        if c in data.columns:
            target_col = c
            break

# Fallback: if no explicit target found, try a sensible default if available
if target_col is None:
    if 'historical_cost_of_ride' in data.columns:
        target_col = 'historical_cost_of_ride'
        print("No explicit target (e.g. Optimal_Price) found — falling back to 'historical_cost_of_ride' as the target.")
    else:
        # helpful error listing available columns
        raise KeyError(
            f"No target column found. Available columns: {list(data.columns)}\n"
            f"Please set COLUMN_MAP = {{'target': '<your_column_name>'}} or rename your column to one of: {target_candidates}"
        )

# Prepare target and features
y = pd.to_numeric(data[target_col], errors='coerce')
X = data.drop(columns=[target_col])

# Quick NaN check for the target
nan_frac = y.isna().mean()
if nan_frac > 0.1:
    print(f"Warning: {nan_frac:.2%} of target values are NaN after coercion. Check your '{target_col}' column.")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data split complete.")
print(f"Training size: {X_train.shape}, Test size: {X_test.shape}")


KeyError: "No target column found. Available columns: ['number_of_riders', 'number_of_drivers', 'location_category', 'customer_loyalty_status', 'number_of_past_rides', 'average_ratings', 'time_of_booking', 'vehicle_type', 'expected_ride_duration', 'historical_cost_of_ride']\nPlease set COLUMN_MAP = {'target': '<your_column_name>'} or rename your column to one of: ['optimal_price', 'optimalprice', 'optimal_price_usd', 'optimal_price_inr', 'price', 'fare', 'predicted_price', 'pred_price', 'dynamic_price']"